# ✝️ Santa Palabra - Entrenamiento Final (Modo ULTRA Ahorro)

Este cuaderno aplica las técnicas más agresivas de ahorro de memoria para funcionar en la T4 gratuita.

**CAMBIOS APLICADOS:**
- Contexto reducido a 512 tokens.
- Gestión de memoria PyTorch optimizada para evitar fragmentación.
- Limpieza agresiva de caché.

**⚠️ PASO OBLIGATORIO:**
1. Ve al menú **Entorno de ejecución** -> **Desconectar y eliminar entorno de ejecución**.
2. Vuelve a conectar a **T4 GPU**.
3. Sube `fine_tuning_data.jsonl`.
4. Ejecuta todo.

In [ ]:
%%capture
import os
# Configuración CRÍTICA para memoria
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["WANDB_DISABLED"] = "true" # Desactivar WandB para ahorrar recursos y evitar logs

import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Instalar Unsloth optimizado para Colab
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch

# AUMENTO LIGERO: 1024 tokens (512 era muy poco y causaba errores de truncado)
max_seq_length = 1024 
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = {'': 0}
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 8, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

dataset = load_dataset("json", data_files="fine_tuning_data.jsonl", split="train")

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import gc

# Limpieza agresiva de memoria antes de empezar
gc.collect()
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1, 
        gradient_accumulation_steps = 8, 
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        gradient_checkpointing = True, # Forzar checkpointing para ahorrar RAM
    ),
)

# Limpieza final pre-entrenamiento
torch.cuda.empty_cache()

trainer.train()

In [ ]:
# Guardar el modelo en formato GGUF para Ollama
model.save_pretrained_gguf("model_gguf", tokenizer, quantization_method = "q4_k_m")